In [ ]:
import json
from collections import Counter
import dateparser

import pandas as pd
import numpy as np
import names

### Example message

```
{
  "id": "12345",
  "type": "GuildMemberJoin",
  "timestamp": "2018-08-25T14:49:47.219+00:00",
  "timestampEdited": null,
  "callEndedTimestamp": null,
  "isPinned": false,
  "content": "Joined the server.",
  "author": {
    "id": "12345678910",
    "name": "Steven",
    "discriminator": "1234",
    "isBot": false,
    "avatarUrl": "avatar.url.com"
  },
  "attachments": [],
  "embeds": [],
  "reactions": []
},
```

In [ ]:
# You can add it more than one word
words = ["bro"] 
chatJSONFileName = "general.json"

In [ ]:
with open(chatJSONFileName) as chatFile:
    data = json.load(chatFile)
    messages = data["messages"]
    
dateDict = dict()

broDict = dict()
for message in messages:
    author = message["author"]["name"]
    
    # Case-insensitive matching
    content = message["content"].lower()
    dateString = message["timestamp"]

    if author not in broDict:
        broDict[author] = Counter()
    
    for word in words:
        if word in content:
            if dateString not in dateDict:
                dateDict[dateString] = np.datetime64(dateparser.parse(dateString).date())
            date = dateDict[dateString]
            broDict[author][date] += 1
broDict

In [ ]:
# Getting the earliest mention the word
earliest = min(d for a in broDict for d in broDict[a])
earliest

In [ ]:
# Latest mention of the word
latest = max(d for a in broDict for d in broDict[a])
latest

In [ ]:
df = pd.DataFrame()
df["dates"] = pd.date_range(earliest, latest)

for author in broDict:
    df[author] = [0] * len(df)
    for date in broDict[author]:
        count = broDict[author][date]
        df.loc[df["dates"] == date, [author]] = count
    df[author] = df[author].cumsum()

In [ ]:
df.set_index("dates", inplace=True)
df

In [ ]:
import bar_chart_race as bcr

In [ ]:
bcr.bar_chart_race(
    df=df,
    filename=f'bro.mp4',
    orientation='h',
    sort='desc',
    n_bars=8,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=3,
    interpolate_period=False,
    label_bars=True,
    bar_size=.95,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='%B %d, %Y',
    period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                      's': f'Total uses of bro: {v.nlargest(8).sum():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},
    perpendicular_bar_func=None,
    period_length=50,
    figsize=(8, 4.5),
    dpi=240,
    cmap='dark12',
    title=f'Messages containing "bro" over time',
    title_size='',
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'DejaVu Sans', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=True)  